## 필요 모듈 설치

In [ ]:
!pip uninstall -y segmentation-models-pytorch
!pip install pretrainedmodels==0.7.4
!pip install efficientnet_pytorch==0.6.3
!pip install timm==0.4.12
!pip install -U git+https://github.com/albu/albumentations --no-cache-dir

!pip install git+https://github.com/qubvel/segmentation_models.pytorch

Found existing installation: segmentation-models-pytorch 0.2.1
Uninstalling segmentation-models-pytorch-0.2.1:
  Successfully uninstalled segmentation-models-pytorch-0.2.1
  Cloning https://github.com/albu/albumentations to /tmp/pip-req-build-2kg8t8jw
  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-2kg8t8jw
  Cloning https://github.com/qubvel/segmentation_models.pytorch to /tmp/pip-req-build-eqvmlphl
  Running command git clone -q https://github.com/qubvel/segmentation_models.pytorch /tmp/pip-req-build-eqvmlphl
  Created wheel for segmentation-models-pytorch: filename=segmentation_models_pytorch-0.2.1-py3-none-any.whl size=88649 sha256=8048c9d051859f16dba9d3a6c1b2ab35e93b8aaded0dfd40161bbee9550f809f
  Stored in directory: /tmp/pip-ephem-wheel-cache-4mzi3toj/wheels/fa/c5/a8/1e8af6cb04a0974db8a4a156ebd2fdd1d99ad2558d3fce49d4
Successfully built segmentation-models-pytorch


## 구글 드라이브 연동

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset
import albumentations as albu
import torch
import numpy as np
import segmentation_models_pytorch as smp

import json

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
workspace_path = '/gdrive/My Drive/Colab Notebooks/SIA'
segmentation_path = os.path.join(workspace_path, 'segmentation_models')

In [ ]:
import sys
sys.path.append(segmentation_path)  # segmentation 소스코드 경로 설정

In [ ]:
#기존 파일 위치 
train_path = os.path.join(workspace_path, 'LV2_training_set')
x_train_dir = os.path.join(train_path, 'images')
y_train_building_dir = os.path.join(train_path, 'labels/building')
y_train_road_dir = os.path.join(train_path, 'labels/road')

train_path = os.path.join(workspace_path, 'LV2_validation_set')
x_valid_dir = os.path.join(train_path, 'images')
y_valid_building_dir = os.path.join(train_path, 'labels/building')
y_valid_road_dir = os.path.join(train_path, 'labels/road')

In [ ]:
#json으로 변환한 파일 위치
train_path = os.path.join(workspace_path, 'training_set')
x_train = os.path.join(train_path, 'images')
y_train = os.path.join(train_path, 'labels')

train_path = os.path.join(workspace_path, 'validation_set')
x_valid = os.path.join(train_path, 'images')
y_valid = os.path.join(train_path, 'labels')


In [ ]:
# helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()

##Create Mask and Image of Building+Road


In [ ]:
# def get_polygon(p):
#   test_list1 = []
#   for i in range(0,len(p),2):
#     test_list2 = [eval(p[i]),eval(p[i+1])]
#     test_list2 = list(map(int, test_list2 ))
#     test_list1.append(test_list2)
#   polygon = np.array(test_list1)
#   return polygon

In [ ]:
# def createLabel(path1, path2, path3):
#   file_list_building = sorted(os.listdir(path1))
#   file_list_road = sorted(os.listdir(path2))

#   file_list = list(set(file_list_building).intersection(file_list_road))
#   print(len(file_list))

#   dict_list_building = []
#   for filename in file_list:
#         if filename.endswith('.json'):
#                 with open(os.path.join(path1, filename), 'r') as f:
#                       dict_list_building.append(json.load(f))

#   dict_list_road = []
#   for filename in file_list:
#         if filename.endswith('.json'):
#                 with open(os.path.join(path2, filename), 'r') as f:
#                       dict_list_road.append(json.load(f))

#   for json_number in range(0,len(file_list)):
#     json_data_building = dict_list_building[json_number]
#     json_data_road = dict_list_road[json_number]

#     mask1 = np.zeros((1024,1024,3),dtype = np.uint8)

#   #road part
#     feature_size_road = len(json_data_road["features"])
#     for i in range(0, feature_size_road):
#       test = json_data_road["features"][i]["properties"]["road_imcoords"].split(',')
#       if test!=['']:
#           polygon = get_polygon(test)
#           cv2.fillPoly(mask1,[polygon], color=(200,200,200))

#   #building part
#     feature_size_building = len(json_data_building["features"])
#     for i in range(0, feature_size_building):
#       test = json_data_building["features"][i]["properties"]["building_imcoords"].split(',')
#       if test!=['']:
#           polygon = get_polygon(test)
#           cv2.fillPoly(mask1,[polygon], color=(255,255,255))



#     file_name = json_data_building["features"][0]["properties"]["image_id"]
#     print(json_number, file_name)
    
#     plt.imsave(os.path.join(path3, file_name), mask1)

In [ ]:
# createLabel(y_valid_building_dir, y_valid_road_dir, y_valid)
# createLabel(y_train_building_dir, y_train_road_dir, y_train)

## Create Mask and Image of Building, Road(사용x)

In [ ]:
# def get_polygon(p):
#   test_list1 = []
#   for i in range(0,len(p),2):
#     test_list2 = [eval(p[i]),eval(p[i+1])]
#     test_list2 = list(map(int, test_list2 ))
#     test_list1.append(test_list2)
#   polygon = np.array(test_list1)
#   return polygon

# def building_Color(test_id):
#   if test_id == '1' :
#     return (0,0,255)
#   elif test_id == '2' :
#     return (0,255,0)
#   elif test_id == '3' :
#     return (255,0,0)
#   elif test_id == '4' :
#     return (255,0,255)
#   elif test_id == '5' :
#     return (0,255,255)
#   elif test_id == '6' : 
#     return (255,255,0)

# def road_Color(test_id):
#   if test_id == '1' :
#     return (97,97,97)
#   elif test_id == '2' :
#     return (97,97,0)
#   elif test_id == '3' :
#     return (0,97,97)
#   elif test_id == '4' :
#     return (97,0,97)
#   elif test_id == '5' :
#     return (97,0,0)
#   elif test_id == '6' : 
#     return (0,97,0)

In [ ]:
# def createFile(path1, path2, path3, path4, path5, path6):
#   file_list_building = sorted(os.listdir(path1))
#   file_list_road = sorted(os.listdir(path2))


#   print("파일리스트 생성 중...")
#   dict_list_building = []
#   for filename in file_list_building:
#         if filename.endswith('.json'):
#                 with open(os.path.join(path1, filename), 'r') as f:
#                       dict_list_building.append(json.load(f))

#   dict_list_road = []
#   for filename in file_list_road:
#         if filename.endswith('.json'):
#                 with open(os.path.join(path2, filename), 'r') as f:
#                       dict_list_road.append(json.load(f))
#   print(len(dict_list_building))
#   print(len(dict_list_road))

#   print("라벨 생성 중...")
#   for json_number in range(0,len(dict_list_building)):
#         json_data = dict_list_building[json_number]
#         mask1 = np.zeros((1024,1024,3),dtype = np.uint8)
#         feature_size_building = len(json_data["features"])
#         for i in range(0, feature_size_building):
#           test = json_data["features"][i]["properties"]["building_imcoords"].split(',')
#           if test!=['']:
#               polygon = get_polygon(test)
#               cv2.fillPoly(mask1,[polygon], color=building_Color(json_data["features"][i]["properties"]["type_id"]))
#               cv2.polylines(mask1,[polygon], True, (255,255,255),3)
#         file_name = json_data["features"][0]["properties"]["image_id"]
#         print(json_number, file_name)
#         plt.imsave(os.path.join(path3, file_name), mask1)

#   for json_number in range(0,len(dict_list_road)):
#         json_data = dict_list_road[json_number]
#         mask1 = np.ones((1024,1024,3),dtype = np.uint8)
#         feature_size_road = len(json_data["features"])
#         for i in range(0, feature_size_road):
#           test = json_data["features"][i]["properties"]["road_imcoords"].split(',')
#           if test!=['']:
#               polygon = get_polygon(test)
#               cv2.fillPoly(mask1,[polygon], color=road_Color(json_data["features"][i]["properties"]["type_id"]))
#         file_name = json_data["features"][0]["properties"]["image_id"]
#         print(json_number, file_name)
#         plt.imsave(os.path.join(path4, file_name), mask1)
  


#   print("이미지 생성 중...")
#   # file_list_building = file_list_building.replace(".json",".png")
#   # file_list_road = file_list_road.replace(".json",".png")

#   for filename in file_list_building:
#       filename = filename.replace(".json",".png")
#       image = cv2.imread(os.path.join(path5, filename))
#       cv2.imwrite(os.path.join(path6, filename),image)




In [ ]:
# createFile(y_train_building_dir, y_train_road_dir, y_train_building, y_train_road, x_train_dir, x_train_building, x_train_road)
# createFile(y_valid_building_dir, y_valid_road_dir, y_valid_building, y_valid_road, x_valid_dir, x_valid_building, x_valid_road)

In [ ]:
# print(len(os.listdir(y_train_building)))
# print(len(os.listdir(y_train_road)))
# print(len(os.listdir(x_train_building)))
# print(len(os.listdir(x_train_road)))

# print(len(os.listdir(y_valid_building)))
# print(len(os.listdir(y_valid_road)))
# print(len(os.listdir(x_valid_building)))
# print(len(os.listdir(x_valid_road)))



##16분할 이미지 생성

In [ ]:
# import random
# from PIL import Image
# import matplotlib.pyplot as plt
# import matplotlib.image as img


# file_list = sorted(os.listdir(x_train))
# for i in range(0,800,16):
#   List = random.sample(file_list, 16)
#   images = []
#   labels = []
#   for j in List :
#     image = Image.open(os.path.join(x_train, j))
#     label = Image.open(os.path.join(y_train, j))
#     for h in range(0,1024,256):
#       for w in range(0,1024,256):
#           box = (w, h, w+256, h+256)
#           a = image.crop(box)  
#           images.append(a)
#           a = label.crop(box)
#           labels.append(a)
#   for k in range(0,16):
#     filename = 'BONUS'+str(i+k)+'.png'
#     mixedImage = []
#     mixedLabel = []
#     mixedImage.append(np.hstack((np.vstack((images[k],images[k+16])),np.vstack((images[k+32],images[k+48])))))
#     mixedImage.append(np.hstack((np.vstack((images[k+64],images[k+80])),np.vstack((images[k+96],images[k+112])))))
#     mixedImage.append(np.hstack((np.vstack((images[k+128],images[k+144])),np.vstack((images[k+160],images[k+176])))))
#     mixedImage.append(np.hstack((np.vstack((images[k+192],images[k+208])),np.vstack((images[k+224],images[k+240])))))
#     image1 = np.vstack((mixedImage[0], mixedImage[1]))
#     image2 = np.vstack((mixedImage[2], mixedImage[3]))
#     image3 = np.hstack((image1,image2))
#     plt.imsave(os.path.join(x_train, filename),image3)

#     mixedLabel.append(np.hstack((np.vstack((labels[k],labels[k+16])),np.vstack((labels[k+32],labels[k+48])))))
#     mixedLabel.append(np.hstack((np.vstack((labels[k+64],labels[k+80])),np.vstack((labels[k+96],labels[k+112])))))
#     mixedLabel.append(np.hstack((np.vstack((labels[k+128],labels[k+144])),np.vstack((labels[k+160],labels[k+176])))))
#     mixedLabel.append(np.hstack((np.vstack((labels[k+192],labels[k+208])),np.vstack((labels[k+224],labels[k+240])))))
#     label1 = np.vstack((mixedLabel[0], mixedLabel[1]))
#     label2 = np.vstack((mixedLabel[2], mixedLabel[3]))
#     label3 = np.hstack((label1,label2))
#     plt.imsave(os.path.join(y_train, filename),label3)

## 데이터로더 정의

In [ ]:
CLASSES = ['building', 'road']

In [ ]:
class Dataset(BaseDataset):
    """CamVid Dataset. Read images, apply augmentation and preprocessing transformations.
    
    Args:
        images_dir (str): path to images folder
        masks_dir (str): path to segmentation masks folder
        class_values (list): values of classes to extract from segmentation mask
        augmentation (albumentations.Compose): data transfromation pipeline 
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing 
            (e.g. noralization, shape manipulation, etc.)
    
    """
    
    CLASSES = ['building', 'road']
    def __init__(
            self, 
            images_dir, 
            masks_dir, 
            classes=None, 
            augmentation=None, 
            preprocessing=None,
            mode=None,
    ):
        self.ids = os.listdir(images_dir)
        self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids]
        self.masks_fps = [os.path.join(masks_dir, image_id) for image_id in self.ids]
        
        # convert str names to class values on masks
        
        # self.class_values = [255, 200]
       
        self.class_values = [self.CLASSES.index(cls.lower()) for cls in classes]

        for i in range(len(self.masks_fps)):
          self.mask_ids = np.unique(cv2.imread(self.masks_fps[i], 0))[1:]
          if len(self.mask_ids) == len(self.class_values):
            break
        
        self.augmentation = augmentation
        self.preprocessing = preprocessing
    
    def __getitem__(self, i):
        
        # read data
        image = cv2.imread(self.images_fps[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.masks_fps[i],0)

        # extract certain classes from mask (e.g. cars)
        masks = [(mask == self.mask_ids[v]) for v in self.class_values]
        mask = np.stack(masks, axis=-1).astype('float')
        

        if mask.shape[-1] != 1:
            background = 1 - mask.sum(axis=-1, keepdims=True)
            mask = np.concatenate((mask, background), axis=-1)
        
        
        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
            
        return image, mask
        
    def __len__(self):
        return len(self.ids)

## 데이터증대

In [ ]:
import albumentations as albu

In [ ]:
def get_training_augmentation():
    train_transform = [

        albu.HorizontalFlip(p=0.5),

        albu.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=1, border_mode=0),

        albu.PadIfNeeded(min_height=320, min_width=320, always_apply=True, border_mode=0),
        albu.RandomCrop(height=320, width=320, always_apply=True),

        albu.IAAAdditiveGaussianNoise(p=0.2),
        albu.IAAPerspective(p=0.5),

        albu.OneOf(
            [
                albu.CLAHE(p=1),
                albu.RandomBrightness(p=1),
                albu.RandomGamma(p=1),
            ],
            p=0.9,
        ),

        albu.OneOf(
            [
                albu.IAASharpen(p=1),
                albu.Blur(blur_limit=3, p=1),
                albu.MotionBlur(blur_limit=3, p=1),
            ],
            p=0.9,
        ),

        albu.OneOf(
            [
                albu.RandomContrast(p=1),
                albu.HueSaturationValue(p=1),
            ],
            p=0.9,
        ),
    ]
    return albu.Compose(train_transform)


def get_validation_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        albu.PadIfNeeded(384, 480)
    ]
    return albu.Compose(test_transform)


def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')


def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)

In [ ]:
dataset = Dataset(x_train, y_train, classes=CLASSES)

image, mask = dataset[350] # get some sample
visualize(
    image=image, 
    road_mask=mask[..., 0].squeeze(),
    building_mask=mask[..., 1].squeeze(),
)

In [ ]:
dataset = Dataset(x_valid, y_valid, classes=CLASSES)

image, mask = dataset[3] # get some sample
visualize(
    image=image, 
    road_mask=mask[..., 0].squeeze(),
    building_mask=mask[..., 1].squeeze(),
)

In [ ]:
augmented_dataset = Dataset(
    x_train, 
    y_train, 
    augmentation=get_training_augmentation(), 
    classes=CLASSES,
)

for i in range(3):
    image, mask = augmented_dataset[400]
    visualize(
        image=image, 
        road_mask=mask[..., 0].squeeze(),
        building_mask=mask[..., 1].squeeze(),
    
    )

## 모델 생성 및 훈련

In [ ]:
ENCODER = 'se_resnext101_32x4d'
ENCODER_WEIGHTS = 'imagenet'
ACTIVATION = 'sigmoid' 
DEVICE = 'cuda'

In [ ]:
model = smp.UnetPlusPlus(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=len(CLASSES)+1, 
    activation=ACTIVATION,
)

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

In [ ]:
train_dataset = Dataset(
    x_train, 
    y_train, 
    augmentation=get_training_augmentation(), 
    preprocessing=get_preprocessing(preprocessing_fn),
    classes=CLASSES,
)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)

/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:262: FutureWarning: IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead
  warnings.warn("IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:385: FutureWarning: This IAAPerspective is deprecated. Please use Perspective instead
  warnings.warn("This IAAPerspective is deprecated. Please use Perspective instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1805: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:232: FutureWarning: IAASharpen is deprecated. Please use Sharpen instead
  warnings.warn("IAASharpen is deprecated. Please use Sharpen instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albument

In [ ]:
valid_dataset = Dataset(
    x_valid, 
    y_valid, 
    preprocessing=get_preprocessing(preprocessing_fn),
    classes=CLASSES,
)

valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False, num_workers=1)

In [ ]:
loss = smp.utils.losses.DiceLoss()
metrics = [
    smp.utils.metrics.IoU(threshold=0.5),
]

optimizer = torch.optim.Adam([dict(params=model.parameters(), lr=0.0001),
])

In [ ]:
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)

In [ ]:
save_dir = os.path.join(workspace_path, 'ckpt')
os.makedirs(save_dir, exist_ok=True)

In [ ]:
max_score = 0

for i in range(0, 400):
    
    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(train_loader)
    valid_logs = valid_epoch.run(valid_loader)

    
     # do something (save model, change lr, etc.)
    if max_score < valid_logs['iou_score']:
        max_score = valid_logs['iou_score']
        torch.save(model, os.path.join(save_dir, 'best_model.pth'))
        print('Model saved!')
      
print('max_score='+str(max_score) )


Epoch: 0
valid: 100%|██████████| 142/142 [01:46<00:00,  1.34it/s, dice_loss - 0.3554, iou_score - 0.6943]
Model saved!
0.6943237483501437

Epoch: 1
valid: 100%|██████████| 142/142 [00:45<00:00,  3.15it/s, dice_loss - 0.2698, iou_score - 0.7351]
Model saved!
0.7351415637933035

Epoch: 2
valid: 100%|██████████| 142/142 [00:45<00:00,  3.14it/s, dice_loss - 0.2198, iou_score - 0.7622]
Model saved!
0.76219800660308

Epoch: 3
valid: 100%|██████████| 142/142 [00:45<00:00,  3.14it/s, dice_loss - 0.1918, iou_score - 0.7758]
Model saved!
0.775809986805412

Epoch: 4
valid: 100%|██████████| 142/142 [00:45<00:00,  3.14it/s, dice_loss - 0.1634, iou_score - 0.7864]
Model saved!
0.7864426468879403

Epoch: 5
valid: 100%|██████████| 142/142 [00:45<00:00,  3.14it/s, dice_loss - 0.1472, iou_score - 0.7953]
Model saved!
0.7953218595242837

Epoch: 6
valid: 100%|██████████| 142/142 [00:45<00:00,  3.13it/s, dice_loss - 0.1427, iou_score - 0.793]
0.7953218595242837

Epoch: 7
valid: 100%|██████████| 142/142 [0

## 이미지 시각화

In [ ]:
best_model = torch.load(os.path.join(save_dir, 'best_model.pth'))

In [ ]:
valid_dataset_vis = Dataset(
    x_valid, y_valid, 
    classes=CLASSES,
)

In [ ]:
for i in range(5):
    n = np.random.choice(len(valid_dataset))
    image, gt_mask = valid_dataset[n]
    gt_mask = gt_mask.squeeze()
    image_vis = valid_dataset_vis[n][0].astype('uint8')

    x_tensor = torch.from_numpy(image).to(DEVICE).unsqueeze(0)
    pr_mask = best_model.predict(x_tensor)
    pr_mask = (pr_mask.squeeze().cpu().numpy().round())

    pr_mask = np.moveaxis(pr_mask, -1, 0)
    gt_mask = np.moveaxis(gt_mask, -1, 0)
    pr_mask = np.moveaxis(pr_mask, -1, 0).squeeze()
    gt_mask = np.moveaxis(gt_mask, -1, 0).squeeze()

    pr_mask*=255
    pr_mask=pr_mask.astype(np.uint8)

    mixed = image_vis.copy()

    hsv_img = cv2.cvtColor(pr_mask, cv2.COLOR_BGR2HSV)

    mask = cv2.inRange(hsv_img, (-20, 100, 100), (20, 255, 255))
    mask_inv = cv2.bitwise_not(mask)
    roi = mixed[0:1024, 0:1024]
    fg = cv2.bitwise_and(pr_mask, pr_mask, mask=mask_inv)
    bg = cv2.bitwise_and(roi, roi, mask=mask)
    mixed[0:1024, 0:1024] = fg + bg

    visualize(
        image=image_vis,
        mixed = mixed,
        gt_mask=gt_mask,
        pr_mask=pr_mask,
        gt_road_mask=gt_mask[..., 0],
        pr_road_mask=pr_mask[..., 0],
        gt_building_mask=gt_mask[..., 1],
        pr_building_mask=pr_mask[..., 1],
    )